In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd 
from time import sleep
import time 
import csv

In [6]:
def connect_to_linkedin ():
    driver.get("https://www.linkedin.com/checkpoint/lg/sign-in-another-account")
    time.sleep(5)
#     username = driver.find_element_by_id("username")
    username = driver.find_element(By.ID,"username")
    username.send_keys("chiheb.jamazi@esprit.tn")
#     pword = driver.find_element_by_id("password")
    pword = driver.find_element(By.ID,"password")
    pword.send_keys("Seawaymn2011&&")
#     driver.find_element_by_xpath("//button[@type='submit']").click()
    driver.find_element(By.XPATH, "//button[@type='submit']").click()

In [7]:
def scrolling_function ():
    SCROLL_PAUSE_TIME = 1.5
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #uncomment to limit the number of scrolls
        #scroll_number += 1
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    return 

In [8]:
Job_post_date =[] 
company_name = []
job_title = []
job_links = []
Number_applicants = []
location = []
working_Place = []
def get_linkedin_jobs (soup):
    
    containers = soup.find_all(class_='jobs-search-results__list-item occludable-update p0 relative ember-view')
    for container in containers:
        try:              
            his=container.find(class_='job-card-container__listed-time job-card-container__footer-item')  
            Job_post_date.append(his.get_text(strip =True))
        except:
            Job_post_date.append('none')
        try:
            text=container.find(class_='job-card-container__link job-card-container__company-name ember-view')
            company_name.append(text.get_text(strip =True))
        except:
            company_name.append('none')
        try:
            desc=container.find(class_='disabled ember-view job-card-container__link job-card-list__title')
            job_title.append(desc.get_text(strip =True))
        except:
            job_title.append('none')  
        try:       
            desc=container.find(class_='disabled ember-view job-card-container__link job-card-list__title')
            descs = 'https://www.linkedin.com'+desc['href']
            job_links.append(descs)
        except:
            job_links.append('none')        

        try:        
            nb_applic=container.find(class_='job-card-container__applicant-count job-card-container__footer-item job-card-container__footer-item--highlighted t-bold inline-flex align-items-center')
            Number_applicants.append(nb_applic.get_text(strip= True))
        except:
            Number_applicants.append('none')

        try:        
            loca=container.find(class_='job-card-container__metadata-item')
            location.append(loca.get_text(strip= True))
        except:
            location.append('none')
        try:        
            working=container.find(class_='job-card-container__metadata-item job-card-container__metadata-item--workplace-type')
            working_Place.append(working.get_text(strip= True))
        except:
            working_Place.append('none')

In [9]:
company = input("Enter your value: ")
page = "https://www.linkedin.com/jobs/search/?geoId=105015875&keywords="+company+"%20jobs"
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# driver = webdriver.Chrome("/usr/local/bin/chromedriver")
driver.set_window_size(900,800)
connect_to_linkedin ()
for numb in range(10):
    sleep(1)
    driver.get(page+'&start='+str(numb*25)) 
    sleep(1)
    scrolling_function ()
    sleep(2)
    src =driver.page_source
    soup =BeautifulSoup(src, 'lxml')
    get_linkedin_jobs (soup)

Enter your value: talan




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_linux64.zip
Driver has been saved in cache [/home/chiheb/.wdm/drivers/chromedriver/linux64/100.0.4896.60]


In [10]:
jobs = pd.DataFrame(
        {'Job_post_date': Job_post_date,
         'company_name': company_name,
         'job_title': job_title, 
         "Number_applicants": Number_applicants,
         "Location": location,
         "working_Place": working_Place,
         "job_links":job_links,

        })

In [39]:
# Jobs_json=jobs.to_dict('records')

### write files into hdfs 

In [103]:
from hdfs import InsecureClient
file_name= company.upper()+'_Linkedin_Jobs_df'
client = InsecureClient('http://localhost:9870/',user='chiheb')
with client.write(file_name, encoding = 'utf-8') as writer:
    jobs.to_csv(writer,index=False)

In [1]:
with client.read(file_name, encoding = 'utf-8') as reader:
  df =  pd.read_csv(reader)

NameError: name 'client' is not defined

In [102]:
df

,Job_post_date,company_name,job_title,Number_applicants,Location,working_Place,job_links
0,3 days ago,TalanConsulting,"Consultant stagiaire - IA, Data & Digital Tran...",none,"Paris, Île-de-France, France",Hybrid,https://www.linkedin.com/jobs/view/2930354074/...
1,3 days ago,Talan,Stage consultant ERP Oracle - H/F,13 applicants,"Paris, Île-de-France, France",none,https://www.linkedin.com/jobs/view/3027408357/...
2,1 day ago,Talan,Développeur Fullstack Java Data - H/F,6 applicants,"Paris, Île-de-France, France",On-site,https://www.linkedin.com/jobs/view/3030347760/...
3,3 weeks ago,Talan,Consultant(e) Data Analytics - H/F,6 applicants,"Lyon, Auvergne-Rhône-Alpes, France",none,https://www.linkedin.com/jobs/view/2982585736/...
4,4 days ago,none,Stage Big Data & Fullstack,7 applicants,"Paris, Île-de-France, France",On-site,https://www.linkedin.com/jobs/view/3024269804/...
...,...,...,...,...,...,...,...
103,1 month ago,Talan,Tech Lead Java/JEE F/H,0 applicants,"Paris, Île-de-France, France",On-site,https://www.linkedin.com/jobs/view/2954232536/...
104,3 weeks ago,Talan,Consultant Senior Talend - H/F,3 applicants,"Nantes, Pays de la Loire, France",On-site,https://www.linkedin.com/jobs/view/2982616387/...
105,2 weeks ago,Talan,Scrum Master - H/F,0 applicants,"Paris, Île-de-France, France",On-site,https://www.linkedin.com/jobs/view/2998010328/...
106,3 days ago,Talan,Consultant(e) en Transformation RH - Conseil /...,11 applicants,"Paris, Île-de-France, France",On-site,https://www.linkedin.com/jobs/view/3026011461/...


### write json file type into hdfs 

In [49]:
# with client.write('model.json', encoding='utf-8') as writer:
#   from json import dump
#   dump(jobsjson, writer)

In [65]:
# fnames = client.list('../chiheb')
# fnames

['model.json', 'modelDf']

In [67]:
# status = client.status('../chiheb/model.json')
# status

{'accessTime': 1650202022263,
 'blockSize': 134217728,
 'childrenNum': 0,
 'fileId': 16397,
 'group': 'supergroup',
 'length': 48629,
 'modificationTime': 1650202022352,
 'owner': 'chiheb',
 'pathSuffix': '',
 'permission': '644',
 'replication': 1,
 'storagePolicy': 0,
 'type': 'FILE'}

In [3]:
import pandas as pd
df = pd.read_csv('accentureLinkedinJobs.csv')


In [5]:
from hdfs import InsecureClient
company='accenture'
file_name= company+'_linkedin_jobs_df'
hdfs_url = "http://localhost:9870/"
hdfs_user = "chiheb"
c = InsecureClient(hdfs_url, user=hdfs_user)
c.makedirs("/user/chiheb/accenture") 
with c.write('accenture/'+file_name, encoding = 'utf-8') as writer:
    df.to_csv(writer,index=False)